In [83]:
!pip install ipython-autotime
%load_ext autotime
!pip install transformers
!pip install -U accelerate
!pip install -U datasets

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 19.6 s (started: 2023-12-16 13:15:13 +00:00)


In [84]:
import pandas as pd
import numpy as np
import torch
import json
import copy
import re
from transformers import pipeline

time: 1.07 ms (started: 2023-12-16 13:15:33 +00:00)


In [237]:
from google.colab import drive
drive.mount('/content/drive')
train_spider_path = '/content/drive/MyDrive/CS559_shared/implementation/spider/train_spider.json'
train_others_path = '/content/drive/MyDrive/CS559_shared/implementation/spider/train_others.json'
spider_tables_json_path = '/content/drive/MyDrive/CS559_shared/implementation/spider/tables.json'
spider_pos_tags_path = '/content/drive/MyDrive/CS559_shared/implementation/2_spider_question_pos_tags.txt'
spider_inferred_type_tags_path = '/content/drive/MyDrive/CS559_shared/implementation/5_spider_inferred_type_tags.txt'
# q_start_index and q_end_index is important
q_start_index = 8000
q_end_index = 8659
spider_inferred_table_tags_path = '/content/drive/MyDrive/CS559_shared/implementation/spider_inferred_table_tags_' + str(q_start_index) + '-' + str(q_end_index) + '.txt'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 4.46 s (started: 2023-12-16 19:02:49 +00:00)


In [238]:
def get_db_id_list_for_all_question(train_spider_path, train_others_path):
  train_spider_json_file = open(train_spider_path)
  train_spider_list = json.load(train_spider_json_file)
  train_others_json_file = open(train_others_path)
  train_others_list = json.load(train_others_json_file)

  train_all = train_spider_list + train_others_list

  spider_question_db_ids = []
  for question_dict in train_all:
    spider_question_db_ids.append(question_dict['db_id'].lower())

  return spider_question_db_ids

time: 2.09 ms (started: 2023-12-16 19:02:53 +00:00)


In [239]:
spider_question_db_ids = get_db_id_list_for_all_question(train_spider_path, train_others_path)
spider_question_db_ids = spider_question_db_ids[q_start_index: q_end_index]
print(spider_question_db_ids)
print(len(spider_question_db_ids))

['scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar'

In [240]:
def retrieve_data_and_tag(path):
    file = open(path, 'r')
    sentences, sentence_tags = [], []
    for line in file:
        # line = re.sub(r"\".*\"","\"xxx\"",line)
        line = re.sub(r"'", "", line)
        word_tag_list = line.split()
        words = []
        tags = []
        for w in word_tag_list:
            item = w.split('/')
            if (len(item[0]) == 0):
                continue

            words.append(item[0])
            if (len(item) < 2):
                print(path, item)
            tags.append(item[1])
        sentences.append(words)
        sentence_tags.append(tags)
        if (len(sentences) != len(sentence_tags)):
            print(line)
    return sentences, sentence_tags

time: 628 µs (started: 2023-12-16 19:02:54 +00:00)


### Reading inferred POS tags

In [241]:
spider_questions_tokens_list, spider_questions_pos_tags_list = retrieve_data_and_tag(spider_pos_tags_path)
spider_questions_tokens_list = spider_questions_tokens_list[q_start_index : q_end_index]
spider_questions_pos_tags_list = spider_questions_pos_tags_list[q_start_index : q_end_index]
print(spider_questions_tokens_list[0])
print(spider_questions_pos_tags_list[0])
print('len spider_questions_tokens_list: ', spider_questions_tokens_list)
print('len spider_questions_pos_tags_list: ', spider_questions_pos_tags_list)
print(len(spider_questions_pos_tags_list))

['how', 'many', 'papers', 'are', 'published', 'in', 'sigir', '?']
['WRB', 'JJ', 'NNS', 'VBP', 'VBN', 'IN', 'NN', '.']
len spider_questions_tokens_list:  [['how', 'many', 'papers', 'are', 'published', 'in', 'sigir', '?'], ['number', 'of', 'papers', 'in', 'sigir', 'conference'], ['what', 'papers', 'have', 'fewer', 'than', '5', 'citations', 'by', 'ACL', 'papers', '?'], ['ACL', 'papers', 'with', 'less', 'than', '5', 'citations'], ['what', 'ACL', 'papers', 'have', 'less', 'than', '5', 'citations', '?'], ['what', 'papers', 'have', 'fewer', 'than', '5', 'citations', 'by', 'ACL', 'papers'], ['What', 'are', 'the', '5', 'most', 'recent', 'papers', 'of', 'Mirella', 'Lapata', '?'], ['What', 'were', 'the', 'papers', 'published', 'at', 'pldi', '2015', '?'], ['Papers', 'from', 'pldi', '2015', 'conference'], ['papers', 'published', 'at', 'pldi', '2015'], ['What', 'are', 'the', 'papers', 'from', 'pldi', '2015', '?'], ['Papers', 'from', 'pldi', '2015', '?'], ['Which', 'papers', 'were', 'accepted', 'in',

### Reading inferred type tags


In [242]:
# reading inferred type tags
spider_questions_tokens_list, spider_questions_type_tags_list = retrieve_data_and_tag(spider_inferred_type_tags_path)
spider_questions_tokens_list = spider_questions_tokens_list[q_start_index : q_end_index]
spider_questions_type_tags_list = spider_questions_type_tags_list[q_start_index : q_end_index]
# examining
print(spider_questions_tokens_list[0])
print(spider_questions_type_tags_list[0])
print('len spider_questions_tokens_list: ', spider_questions_tokens_list)
print('len spider_questions_type_tags_list: ', spider_questions_type_tags_list)
print(len(spider_questions_type_tags_list))

['how', 'many', 'papers', 'are', 'published', 'in', 'sigir', '?']
['O', 'O', 'TABLE', 'O', 'TABLEREF', 'COND', 'VALUE', 'O']
len spider_questions_tokens_list:  [['how', 'many', 'papers', 'are', 'published', 'in', 'sigir', '?'], ['number', 'of', 'papers', 'in', 'sigir', 'conference'], ['what', 'papers', 'have', 'fewer', 'than', '5', 'citations', 'by', 'ACL', 'papers', '?'], ['ACL', 'papers', 'with', 'less', 'than', '5', 'citations'], ['what', 'ACL', 'papers', 'have', 'less', 'than', '5', 'citations', '?'], ['what', 'papers', 'have', 'fewer', 'than', '5', 'citations', 'by', 'ACL', 'papers'], ['What', 'are', 'the', '5', 'most', 'recent', 'papers', 'of', 'Mirella', 'Lapata', '?'], ['What', 'were', 'the', 'papers', 'published', 'at', 'pldi', '2015', '?'], ['Papers', 'from', 'pldi', '2015', 'conference'], ['papers', 'published', 'at', 'pldi', '2015'], ['What', 'are', 'the', 'papers', 'from', 'pldi', '2015', '?'], ['Papers', 'from', 'pldi', '2015', '?'], ['Which', 'papers', 'were', 'accepted'

### Reading inferred table tags

In [243]:
# reading inferred table tags
spider_questions_tokens_list, spider_questions_table_tags_list = retrieve_data_and_tag(spider_inferred_table_tags_path)
# you may need comment out the 2 lines below
# spider_questions_tokens_list = spider_questions_tokens_list[q_start_index : q_end_index]
# spider_questions_table_tags_list = spider_questions_table_tags_list[q_start_index : q_end_index]

# examining
print(spider_questions_tokens_list[0])
print(spider_questions_table_tags_list[0])
print('len spider_questions_tokens_list: ', spider_questions_tokens_list)
print('len spider_questions_table_tags_list: ', spider_questions_table_tags_list)
print(len(spider_questions_table_tags_list))

['how', 'many', 'papers', 'are', 'published', 'in', 'sigir', '?']
['O', 'O', 'paper', 'O', 'paper', 'O', 'venue', 'O']
len spider_questions_tokens_list:  [['how', 'many', 'papers', 'are', 'published', 'in', 'sigir', '?'], ['number', 'of', 'papers', 'in', 'sigir', 'conference'], ['what', 'papers', 'have', 'fewer', 'than', '5', 'citations', 'by', 'ACL', 'papers', '?'], ['ACL', 'papers', 'with', 'less', 'than', '5', 'citations'], ['what', 'ACL', 'papers', 'have', 'less', 'than', '5', 'citations', '?'], ['what', 'papers', 'have', 'fewer', 'than', '5', 'citations', 'by', 'ACL', 'papers'], ['What', 'are', 'the', '5', 'most', 'recent', 'papers', 'of', 'Mirella', 'Lapata', '?'], ['What', 'were', 'the', 'papers', 'published', 'at', 'pldi', '2015', '?'], ['Papers', 'from', 'pldi', '2015', 'conference'], ['papers', 'published', 'at', 'pldi', '2015'], ['What', 'are', 'the', 'papers', 'from', 'pldi', '2015', '?'], ['Papers', 'from', 'pldi', '2015', '?'], ['Which', 'papers', 'were', 'accepted', 'in'

In [244]:
def create_spider_table_info_list(spider_tables_json_path):
  spider_table_json_file = open(spider_tables_json_path)
  spider_table_info_list = json.load(spider_table_json_file)

  # storing spider table and infos as a dict
  spider_tables_info_dict={}
  for db_dict in spider_table_info_list:
    if db_dict['db_id'] not in spider_tables_info_dict:
      lower_cased_db_dict = {}
      # lowering items of "column_names" list
      lower_cased_db_dict['column_names'] = []
      for col_name in db_dict['column_names']:
        col_name = [col_name[0], col_name[1].lower()]
        lower_cased_db_dict['column_names'].append(col_name)

      # lowering items of "column_names_original" list
      lower_cased_db_dict['column_names_original'] = []
      for col_name_org in db_dict['column_names_original']:
        col_name_org = [col_name_org[0], col_name_org[1].lower()]
        lower_cased_db_dict['column_names_original'].append(col_name_org)

      lower_cased_db_dict['column_types'] = db_dict['column_types'] # adding column types which are already lowered
      lower_cased_db_dict['db_id'] = db_dict['db_id'].lower() # adding db_id
      lower_cased_db_dict['foreign_keys'] = db_dict['foreign_keys']
      lower_cased_db_dict['primary_keys'] = db_dict['primary_keys']

      # lowering items of 'table_names" list
      lower_cased_db_dict['table_names'] = [tname.lower() for tname in db_dict['table_names']]
      # lowering items of 'table_names_original" list
      lower_cased_db_dict['table_names_original'] = [tname.lower() for tname in db_dict['table_names_original']]


      # spider_tables_info_dict[db_dict['db_id'].lower()] = db_dict
      spider_tables_info_dict[db_dict['db_id'].lower()] = lower_cased_db_dict

  return spider_tables_info_dict


time: 1.74 ms (started: 2023-12-16 19:02:55 +00:00)


In [245]:
def get_db_tables_and_columns(spider_tables_info_dict, db_id):
  db_table_columns_dict = {}
  table_names = spider_tables_info_dict[db_id]['table_names_original']
  for table_index in range(len(table_names)):
    table_name = table_names[table_index]
    db_table_columns_dict[table_name] = [col_item[1] for col_item in spider_tables_info_dict[db_id]['column_names_original'] if col_item[0] == table_index ]

  # table_names are keys and table columns are values as list
  return db_table_columns_dict

time: 639 µs (started: 2023-12-16 19:02:55 +00:00)


In [246]:
spider_tables_info_dict = create_spider_table_info_list(spider_tables_json_path)
print(spider_tables_info_dict)

{'perpetrator': {'column_names': [[-1, '*'], [0, 'perpetrator id'], [0, 'people id'], [0, 'date'], [0, 'year'], [0, 'location'], [0, 'country'], [0, 'killed'], [0, 'injured'], [1, 'people id'], [1, 'name'], [1, 'height'], [1, 'weight'], [1, 'home town']], 'column_names_original': [[-1, '*'], [0, 'perpetrator_id'], [0, 'people_id'], [0, 'date'], [0, 'year'], [0, 'location'], [0, 'country'], [0, 'killed'], [0, 'injured'], [1, 'people_id'], [1, 'name'], [1, 'height'], [1, 'weight'], [1, 'home town']], 'column_types': ['text', 'number', 'number', 'text', 'number', 'text', 'text', 'number', 'number', 'number', 'text', 'number', 'number', 'text'], 'db_id': 'perpetrator', 'foreign_keys': [[2, 9]], 'primary_keys': [1, 9], 'table_names': ['perpetrator', 'people'], 'table_names_original': ['perpetrator', 'people']}, 'college_2': {'column_names': [[-1, '*'], [0, 'building'], [0, 'room number'], [0, 'capacity'], [1, 'department name'], [1, 'building'], [1, 'budget'], [2, 'course id'], [2, 'title']

In [247]:
# example for get_db_tables_and_columns
first_question_db_id = spider_question_db_ids[0].lower()
print('first_question_db_id: ', first_question_db_id)
first_question_db_tables_columns = get_db_tables_and_columns(spider_tables_info_dict, first_question_db_id)
print('first_question_db_tables_columns: ', first_question_db_tables_columns)

first_question_db_id:  scholar
first_question_db_tables_columns:  {'venue': ['venueid', 'venuename'], 'author': ['authorid', 'authorname'], 'dataset': ['datasetid', 'datasetname'], 'journal': ['journalid', 'journalname'], 'keyphrase': ['keyphraseid', 'keyphrasename'], 'paper': ['paperid', 'title', 'venueid', 'year', 'numciting', 'numcitedby', 'journalid'], 'cite': ['citingpaperid', 'citedpaperid'], 'paperdataset': ['paperid', 'datasetid'], 'paperkeyphrase': ['paperid', 'keyphraseid'], 'writes': ['paperid', 'authorid']}
time: 862 µs (started: 2023-12-16 19:02:55 +00:00)


### Serialization of input

In [248]:
# Since we have table tags for each token, we can serialize input
# using type tags and columns of related tables
counter = 0
def create_serialized_input(question_input_token_list, question_type_tag_list, question_table_tag_list, spider_tables_info_dict, spider_question_db_ids):
  serialized_inputs = []

  for index in range(len(question_input_token_list)):
    # extraction questioin tokens, type tags and table tags
    question_tokens = question_input_token_list[index]
    question_type_tags = question_type_tag_list[index]
    question_table_tags = question_table_tag_list[index]
    # print('index: ', index)

    # extraction related database table and columsn information
    db_id = spider_question_db_ids[index]
    db_tables_columns =  get_db_tables_and_columns(spider_tables_info_dict, db_id)
    print('db_id: ', db_id)
    print(f"DB Tables: {db_tables_columns}")
    # print('db_tables_columns: ', db_tables_columns)

    serialized_input = copy.deepcopy(question_tokens)
    serialized_input.append("[TTAG]")
    serialized_input = serialized_input + question_type_tags
    serialized_input.append("[SEP]")

    related_tables = []
    colums_to_be_added = []
    for table_tag in question_table_tags:
      if table_tag != 'O' and table_tag not in related_tables:
        related_tables.append(table_tag)
        print('table_tag: ', table_tag)
        table_columns = db_tables_columns.get(table_tag, []) # if key doesn't exist give empty list
        print('table_columns: ', table_columns)
        for col in table_columns:
          if col not in colums_to_be_added:
            colums_to_be_added.append(col.lower())

    if(len(colums_to_be_added) == 0):
      global counter
      counter += 1
      #print(question_table_tags)

    # adding unique related columns
    serialized_input = serialized_input + colums_to_be_added

    serialized_inputs.append(serialized_input)


  return serialized_inputs

time: 1.22 ms (started: 2023-12-16 19:02:55 +00:00)


In [249]:
serialized_inputs_token_list = create_serialized_input(spider_questions_tokens_list, spider_questions_type_tags_list, spider_questions_table_tags_list, spider_tables_info_dict, spider_question_db_ids)
print(serialized_inputs_token_list[0])
print(counter)

db_id:  scholar
DB Tables: {'venue': ['venueid', 'venuename'], 'author': ['authorid', 'authorname'], 'dataset': ['datasetid', 'datasetname'], 'journal': ['journalid', 'journalname'], 'keyphrase': ['keyphraseid', 'keyphrasename'], 'paper': ['paperid', 'title', 'venueid', 'year', 'numciting', 'numcitedby', 'journalid'], 'cite': ['citingpaperid', 'citedpaperid'], 'paperdataset': ['paperid', 'datasetid'], 'paperkeyphrase': ['paperid', 'keyphraseid'], 'writes': ['paperid', 'authorid']}
table_tag:  paper
table_columns:  ['paperid', 'title', 'venueid', 'year', 'numciting', 'numcitedby', 'journalid']
table_tag:  venue
table_columns:  ['venueid', 'venuename']
db_id:  scholar
DB Tables: {'venue': ['venueid', 'venuename'], 'author': ['authorid', 'authorname'], 'dataset': ['datasetid', 'datasetname'], 'journal': ['journalid', 'journalname'], 'keyphrase': ['keyphraseid', 'keyphrasename'], 'paper': ['paperid', 'title', 'venueid', 'year', 'numciting', 'numcitedby', 'journalid'], 'cite': ['citingpaper

In [250]:
print(serialized_inputs_token_list[15])

['List', 'of', 'papers', 'in', 'pldi', '2015', '[TTAG]', 'O', 'O', 'TABLE', 'COND', 'VALUE', 'VALUE', '[SEP]', 'paperid', 'title', 'venueid', 'year', 'numciting', 'numcitedby', 'journalid', 'venuename']
time: 414 µs (started: 2023-12-16 19:02:55 +00:00)


In [251]:
inputs = [" ".join(in_token_list) for in_token_list in serialized_inputs_token_list]
print('total input number: ', len(inputs))
print(inputs[:3])

total input number:  659
['how many papers are published in sigir ? [TTAG] O O TABLE O TABLEREF COND VALUE O [SEP] paperid title venueid year numciting numcitedby journalid venuename', 'number of papers in sigir conference [TTAG] O O TABLE COND VALUE TABLE [SEP] paperid title venueid year numciting numcitedby journalid venuename', 'what papers have fewer than 5 citations by ACL papers ? [TTAG] O TABLE O COND O O TABLE O VALUE TABLE O [SEP] paperid title venueid year numciting numcitedby journalid venuename']
time: 2.27 ms (started: 2023-12-16 19:02:55 +00:00)


In [252]:
# Load the model from the Hugging Face Hub
model_schema_col_tagger = pipeline(model="epolat/t5_finetuned-dbtag-value-part")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


time: 2.99 s (started: 2023-12-16 19:02:55 +00:00)


In [253]:
# an example
example_index = 0
example_input = inputs[example_index]
print(example_input)
example_nonserialized_input = spider_questions_tokens_list[example_index]
print('example_nonserialized_input: ', example_nonserialized_input)
print('len of example_nonserialized_input: ', len(example_nonserialized_input))
predicted_output = model_schema_col_tagger(example_input, min_new_tokens=len(example_nonserialized_input), max_new_tokens=len(example_nonserialized_input)+1)
generated_text = predicted_output[0]['generated_text']
print('generated_text: ', generated_text)
print('token len of generated_text: ', len(generated_text.split(" ")))
predicted_output = model_schema_col_tagger(example_input)
generated_text = predicted_output[0]['generated_text']
print('generated_text 2: ', generated_text)
print('token len of generated_text 2: ', len(generated_text.split(" ")))

how many papers are published in sigir ? [TTAG] O O TABLE O TABLEREF COND VALUE O [SEP] paperid title venueid year numciting numcitedby journalid venuename
example_nonserialized_input:  ['how', 'many', 'papers', 'are', 'published', 'in', 'sigir', '?']
len of example_nonserialized_input:  8
generated_text:  O O O O O O venuename O
token len of generated_text:  8
generated_text 2:  O O O O O O venuename O
token len of generated_text 2:  8
time: 605 ms (started: 2023-12-16 19:02:59 +00:00)


In [254]:
generated_text_token_list = []
for idx in range(len(inputs)):
  input = inputs[idx]
  # print(input)
  nonserialized_input_tokens = spider_questions_tokens_list[idx]
  # print(nonserialized_input_tokens)
  # predicted_output = model_schema_col_tagger(input, min_length=len(nonserialized_input_tokens), max_length=len(nonserialized_input_tokens)+1)
  predicted_output = model_schema_col_tagger(input)
  generated_text_tokens = predicted_output[0]['generated_text'].split()
  generated_text_token_list.append(generated_text_tokens[: ])



time: 3min 38s (started: 2023-12-16 19:02:59 +00:00)


In [255]:
# writing whole dataset into text file


path = '/content/drive/MyDrive/CS559_shared/implementation/spider_inferred_column_tags_' +  str(q_start_index) + '-' + str(q_end_index) + '.txt'
with open(path, 'w') as file_handle:
  for input_tokens, generated_text_tokens in zip(spider_questions_tokens_list, generated_text_token_list):
    for ind in range(len(input_tokens)):
      if ind < len(generated_text_tokens):
        token_with_type = input_tokens[ind]+'/'+generated_text_tokens[ind]+ '/ '
      else:
        token_with_type = input_tokens[ind]+'/O/ '
      # print(token_with_type, end='')
      file_handle.write(token_with_type)
    file_handle.write('\n')
    # print()



time: 10.9 ms (started: 2023-12-16 19:06:37 +00:00)


time: 17.5 ms (started: 2023-12-16 19:06:37 +00:00)
